# BSGP model Tutorial

In [13]:
import numpy as np
from bsgp.models import RegressionModel
import argparse
tf.compat.v1.disable_eager_execution()
import json
from sklearn.model_selection import train_test_split
from torch.utils.data import  TensorDataset
from scipy.stats import bernoulli as np_bern
from bsgp.dgp_model import DGP
from scipy.stats import norm
from scipy.special import logsumexp
from bsgp.kernels import SquaredExponential as BgpSE
from bsgp.likelihoods import Gaussian
import tensorflow as tf

PRIORS = ['uniform', 'normal', 'determinantal', 'strauss']
import torch

Load the Boston dataset as an example

In [3]:
from run_regression import create_dataset

In [9]:
X_train, Y_train,  X_test, Y_test, Y_train_mean, Y_train_std = create_dataset("Boston", 0)
print('Shape of X_train: %d x %d'%(X_train.shape[0],X_train.shape[1]))
print('Shape of Y_train: %d x %d'%(Y_train.shape[0],Y_train.shape[1]))
print('Shape of X_test:  %d x %d'%(X_test.shape[0],X_test.shape[1]))
print('Shape of Y_test:  %d x %d'%(Y_test.shape[0],Y_test.shape[1]))

Shape of X_train: 404 x 13
Shape of Y_train: 404 x 1
Shape of X_test:  102 x 13
Shape of Y_test:  102 x 1


## Create RegressionModel

Now set the properties (ARGS) of the model (the ones that are specified with the command line arguments)

In [14]:
prior_type = 'normal' # prior / strauss / determinantal / uniform
class ARGS:
    num_inducing = 100
    iterations = 10000
    minibatch_size = 100
    window_size = 64
    num_posterior_samples = 100
    posterior_sample_spacing = 50
    full_cov = True
    n_layers = 1
    prior_type = None
    logdir = '/tmp/'
ARGS = ARGS()
ARGS.num_inducing = 100
ARGS.minibatch_size = 10
ARGS.iterations = 1000
ARGS.n_layers = 1
ARGS.num_posterior_samples = 100
ARGS.prior_type = prior_type
ARGS.full_cov = False
ARGS.posterior_sample_spacing = 32
print('Number of inducing points: %d' % ARGS.num_inducing)

Number of inducing points: 100


Fitting the RegressionModel

In [15]:
lik = Gaussian(np.var(Y_train, 0)) # define a Likelihood

In [19]:
Y = Y_train
if len(Y.shape) == 1:
    Y = Y[:, None]
Y.shape

(404, 1)

In [20]:
kerns = []
for i in range(ARGS.n_layers):
    output_dim = 196 if i >= 1 and X_train.shape[1] > 700 else X_train.shape[1]
    lengthscales = float(min(X_train.shape[1], output_dim))
    print('#Layer: %d - Lengthscales: '%(i), lengthscales)
    kerns.append(BgpSE(output_dim, ARD=True, lengthscales=lengthscales**0.5))

#Layer: 0 - Lengthscales:  13.0


In [22]:
mb_size = ARGS.minibatch_size if X_train.shape[0] > ARGS.minibatch_size else X_train.shape[0]
print('Minibatch size: %d'%(mb_size))

Minibatch size: 10


Stochastic Hamiltonian Monte Carlo Sampling

In [23]:
X_placeholder = tf.compat.v1.placeholder(tf.float64, shape=[None, X_train.shape[1]])
Y_placeholder = tf.compat.v1.placeholder(tf.float64, shape=[None, Y_train.shape[1]])

In [25]:
data_iter = 0
def get_minibatch():
    N = X_train.shape[0]
    minibatch_size = mb_size
    assert N >= minibatch_size
    if N == minibatch_size:
        return X_train, Y_train

    if N < data_iter + minibatch_size:
        shuffle = np.random.permutation(N)
        X = X_train[shuffle, :]
        Y = Y_train[shuffle, :]
        data_iter = 0

    X_batch = X[data_iter:data_iter + minibatch_size, :]
    Y_batch = Y[data_iter:data_iter + minibatch_size, :]
    data_iter += minibatch_size
    return X_batch, Y_batch

In [ ]:
def sghmc_step():
    X_batch, Y_batch = get_minibatch()
    feed_dict = {X_placeholder: X_batch, Y_placeholder: Y_batch}
    self.session.run(self.burn_in_op, feed_dict=feed_dict)
    for j in range(10):
        X_batch, Y_batch = self.get_minibatch()
        feed_dict = {self.X_placeholder: X_batch, self.Y_placeholder: Y_batch}
        self.session.run(self.burn_in_op, feed_dict=feed_dict)
        self.session.run((self.sample_op), feed_dict=feed_dict)

    values = self.session.run((self.vars))
    sample = {}
    for var, value in zip(self.vars, values):
        sample[var] = value
    self.window.append(sample)
    if len(self.window) > self.window_size:
        self.window = self.window[-self.window_size:]

In [ ]:
global_step = 0
for _ in range(self.ARGS.iterations):
    global_step += 1
    self.model.sghmc_step()
    if ARGS.prior_type == "determinantal":
        model.reset_Lm()
        self.model.train_hypers() if hasattr(self.model, 'hyper_train_op') else None
        if _ % 250 == 1:
            marginal_ll = self.model.print_sample_performance()
            # writer.add_scalar('optimisation/marginal_likelihood', marginal_ll*len(X), self.global_step)
            print('TRAIN | iter = %6d      sample marginal LL = %5.2f' % (_, marginal_ll))
            # Test with previous samples with Xtest and Ytest are both not None
            if not (Xtest is None or Ytest is None or Ystd is None):
                ms, vs = self.model.predict_y(Xtest, len(self.model.window), posterior=False)
                logps = norm.logpdf(np.repeat(Ytest[None, :, :]*Ystd, len(self.model.window), axis=0), ms*Ystd, np.sqrt(vs)*Ystd)
                mnll = -np.mean(logsumexp(logps, axis=0) - np.log(len(self.model.window)))
                # writer.add_scalar('test/predictive_nloglikelihood', mnll, self.global_step)
                print('TEST  | iter = %6d       MNLL = %5.2f' % (_, mnll))